In [1]:
import numpy as np
import tensorflow as tf
import re
import time

In [4]:
lines = open('movie_lines.txt',encoding='utf-8',errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt',encoding='utf-8',errors='ignore').read().split('\n')

In [24]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line)==5:
        id2line[_line[0]] = _line[4]

In [25]:
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [9]:
conversations

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

In [15]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(_conversation.split(','))

In [16]:
conversations_ids

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [26]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+ 1]])

In [27]:
questions

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...',
 "C'esc ma tete. This is my head",
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have n

In [28]:
answers

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.',
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 "That's because it's such a nice one.",
 'Forget French.',
 "Well, there's someone I think might be --",
 'Where?',
 "I 

In [57]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"\'ll"," will",text)
    text = re.sub(r"\'ve'"," have",text)
    text = re.sub(r"\'re"," are",text)
    text = re.sub(r"\'d"," would",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","cannot",text)
    text = re.sub(r'[^\w\s]','', text) 
    return text

In [58]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [60]:
answers

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.',
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 "That's because it's such a nice one.",
 'Forget French.',
 "Well, there's someone I think might be --",
 'Where?',
 "I 

In [59]:
clean_answers

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'okay then how bout we try out some french cuisine  saturday  night',
 'forget it',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do',
 'right  see  you are ready for the quiz',
 'i dont want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone',
 'that is because its such a nice one',
 'forget french',
 'well theres someone i think might be ',
 'where',
 'i counted on you to help my cause you 

In [61]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word]+=1
for ans in clean_answers:
    for word in ans.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word]+=1

In [62]:
word2count

{'can': 16006,
 'we': 38605,
 'make': 6761,
 'this': 33902,
 'quick': 361,
 'roxanne': 1,
 'korrine': 1,
 'and': 65628,
 'andrew': 56,
 'barrett': 20,
 'are': 54712,
 'having': 1219,
 'an': 10179,
 'incredibly': 60,
 'horrendous': 4,
 'public': 369,
 'break': 918,
 'up': 16582,
 'on': 27578,
 'the': 140764,
 'quad': 2,
 'again': 3330,
 'well': 14181,
 'i': 197067,
 'thought': 4564,
 'would': 19787,
 'start': 1658,
 'with': 24991,
 'pronunciation': 2,
 'if': 18978,
 'that': 67643,
 'is': 78552,
 'okay': 6189,
 'you': 210169,
 'not': 32553,
 'hacking': 18,
 'gagging': 9,
 'spitting': 16,
 'part': 1443,
 'please': 3437,
 'asking': 748,
 'me': 46219,
 'out': 18909,
 'so': 19111,
 'cute': 281,
 'what': 55652,
 'your': 29950,
 'name': 3157,
 'no': 28776,
 'its': 26836,
 'my': 29731,
 'fault': 515,
 'didnt': 8837,
 'have': 31128,
 'a': 102064,
 'proper': 141,
 'introduction': 19,
 'cameron': 35,
 'thing': 5827,
 'am': 37926,
 'at': 15301,
 'mercy': 75,
 'of': 56312,
 'particularly': 111,
 'hi

In [64]:
thresh = 20
questionwords2int = {}
word_number = 0
for word,count in word2count.items():
    if count>thresh:
        questionwords2int[word] = word_number
        word_number+=1
answerswords2int = {}
word_number = 0
for word,count in word2count.items():
    if count>thresh:
        answerswords2int[word] = word_number
        word_number+=1

In [65]:
questionwords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'are': 7,
 'having': 8,
 'an': 9,
 'incredibly': 10,
 'public': 11,
 'break': 12,
 'up': 13,
 'on': 14,
 'the': 15,
 'again': 16,
 'well': 17,
 'i': 18,
 'thought': 19,
 'would': 20,
 'start': 21,
 'with': 22,
 'if': 23,
 'that': 24,
 'is': 25,
 'okay': 26,
 'you': 27,
 'not': 28,
 'part': 29,
 'please': 30,
 'asking': 31,
 'me': 32,
 'out': 33,
 'so': 34,
 'cute': 35,
 'what': 36,
 'your': 37,
 'name': 38,
 'no': 39,
 'its': 40,
 'my': 41,
 'fault': 42,
 'didnt': 43,
 'have': 44,
 'a': 45,
 'proper': 46,
 'cameron': 47,
 'thing': 48,
 'am': 49,
 'at': 50,
 'mercy': 51,
 'of': 52,
 'particularly': 53,
 'breed': 54,
 'loser': 55,
 'sister': 56,
 'cannot': 57,
 'date': 58,
 'until': 59,
 'she': 60,
 'does': 61,
 'why': 62,
 'mystery': 63,
 'used': 64,
 'to': 65,
 'be': 66,
 'really': 67,
 'popular': 68,
 'when': 69,
 'started': 70,
 'high': 71,
 'school': 72,
 'then': 73,
 'it': 74,
 'was': 75,
 'just': 7

In [66]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionwords2int[token] = len(questionwords2int)+1
for token in tokens:
    answerswords2int[token] = len(answerswords2int)+1

In [67]:
answersints2word = {w_i:w for w,w_i in answerswords2int.items()}

In [68]:
for i in range(len(clean_answers)):
    clean_answers[i]+=' <EOS>'

In [69]:
clean_answers

['well i thought we would start with pronunciation if that is okay with you <EOS>',
 'not the hacking and gagging and spitting part  please <EOS>',
 'okay then how bout we try out some french cuisine  saturday  night <EOS>',
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does <EOS>',
 'seems like she could get a date easy enough <EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>',
 'right  see  you are ready for the quiz <EOS>',
 'i dont want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone <EOS>',
 'that is because its such a nice one <EOS>',
 'forget french <EOS>',
 'well t

In [75]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionwords2int:
            ints.append(questionwords2int['<OUT>'])
        else:
            ints.append(questionwords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

answers_into_int

In [77]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1,25+1):
    for i in enumerate(questions_into_int):
        if len(i[1])==length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [108]:
sorted_clean_questions

[[47],
 [62],
 [123],
 [147],
 [135],
 [39],
 [175],
 [39],
 [181],
 [182],
 [220],
 [36],
 [62],
 [135],
 [62],
 [110],
 [371],
 [132],
 [296],
 [209],
 [36],
 [221],
 [181],
 [26],
 [62],
 [181],
 [8362],
 [451],
 [249],
 [181],
 [192],
 [8362],
 [8362],
 [662],
 [97],
 [8362],
 [8362],
 [36],
 [39],
 [36],
 [8362],
 [147],
 [90],
 [36],
 [36],
 [8362],
 [630],
 [39],
 [39],
 [933],
 [819],
 [1106],
 [39],
 [231],
 [69],
 [209],
 [39],
 [142],
 [8362],
 [209],
 [1098],
 [1098],
 [1098],
 [1098],
 [209],
 [336],
 [149],
 [26],
 [92],
 [662],
 [36],
 [1125],
 [1240],
 [1482],
 [8362],
 [1487],
 [209],
 [1527],
 [36],
 [36],
 [209],
 [1575],
 [1575],
 [1575],
 [1575],
 [1575],
 [1575],
 [26],
 [662],
 [67],
 [662],
 [231],
 [97],
 [1604],
 [1575],
 [1575],
 [1575],
 [8362],
 [1575],
 [1575],
 [67],
 [1644],
 [667],
 [1760],
 [209],
 [38],
 [209],
 [1206],
 [209],
 [1323],
 [209],
 [17],
 [123],
 [36],
 [62],
 [1810],
 [1742],
 [209],
 [1818],
 [209],
 [209],
 [1792],
 [221],
 [1133],
 [

In [109]:
sorted_clean_answers

[[15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8362,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61,
  8361],
 [8362,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81,
  8361],
 [102, 8361],
 [1504, 77, 101, 1524, 33, 149, 602, 8361],
 [27, 153, 227, 3, 6168, 8361],
 [26, 27, 7, 160, 253, 65, 1257, 97, 65, 607, 8361],
 [726, 134, 8361],
 [27, 238, 133, 193, 225, 74, 8361],
 [195, 8361],
 [20, 27, 124, 606, 32, 45, 1488, 47, 8361],
 [41,
  2523,
  157,
  210,
  78,
  45,
  103,
  1097,
  50,
  956,
  15,
  8362,
  144,
  218,
  515,
  8361],
 [111, 8362, 93, 45, 271, 8361],
 [179, 75, 77, 45, 272, 273, 8361],
 [279,
  24,
  18,
  280,
  18,
  20,
  110,
  128,
  281,
  76,
  116,
  278,
  267,
  75,
  264,
  74,
  5,
  18,
  282,
  283,
  284,
  93,
  8362,
  152,
  5,
  41,
  8362,
  8362,
  285,
  8361],
 [116, 60, 243, 4463, 378, 296, 8361],
 [836

In [119]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,Dropout,Dense
from tensorflow.keras.models import Sequential

In [173]:
padded_seq_questions = pad_sequences(sorted_clean_questions,maxlen=553)
padded_seq_answers = pad_sequences(sorted_clean_answers)

In [174]:
padded_seq_questions = padded_seq_questions.reshape(204079,553,1)
padded_seq_answers = padded_seq_answers.reshape(204079,553,1)
#print(padded_seq_questions.shape)
#padded_seq_answers.shape

In [176]:
model = Sequential()
model.add(LSTM(units = 64, return_sequences=True, input_shape = (553,1)))
model.add(Dropout(0.1))
model.add(LSTM(units = 50,return_sequences = True))
model.add(Dropout(0.1))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['SparseCategoricalAccuracy'])

In [177]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 553, 64)           16896     
_________________________________________________________________
dropout_26 (Dropout)         (None, 553, 64)           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 553, 50)           23000     
_________________________________________________________________
dropout_27 (Dropout)         (None, 553, 50)           0         
_________________________________________________________________
dense_15 (Dense)             (None, 553, 1)            51        
Total params: 39,947
Trainable params: 39,947
Non-trainable params: 0
_________________________________________________________________


In [178]:
padded_seq_questions.shape

(204079, 553, 1)

In [179]:
padded_seq_answers.shape

(204079, 553, 1)

In [ ]:
model.fit(padded_seq_questions,padded_seq_answers,epochs=50,batch_size=32,verbose=1)

Epoch 1/50
 462/6378 [=>............................] - ETA: 37:51 - loss: 4.2071e-06 - sparse_categorical_accuracy: 0.9788